In [ ]:
using Pkg

# Pkg.add("NNlib")
# Pkg.add("DataFrames")
# Pkg.add("ResumableFunctions")

In [1]:
using CSV
using NNlib
using Plots
using DataFrames
using Distributions
using ResumableFunctions

using Flux

┌ Error: This version of CUDA.jl only supports NVIDIA drivers for CUDA 11.x or higher (yours is for CUDA 9.1.0)
└ @ CUDA C:\Users\hurub\.julia\packages\CUDA\s0e3j\src\initialization.jl:64


In [ ]:
# smoothing a vector's values by applying moving average
function moving_average(data; window_size = 100)
    ma = []

    for i = 1 : size(data, 1)
        if i < window_size
            append!(ma, [sum(data[1:i]) / i])
        else
            append!(ma, [sum(data[i-window_size+1:i]) / window_size])
        end
    end

    ma
end

In [ ]:
parkinson_data = CSV.read("./filtered_data.csv", DataFrame)

parkinson_copy = deepcopy(parkinson_data)

;

In [ ]:
window_size = 40

parkinson_copy[!, "AccV"]  = moving_average(parkinson_copy[!, "AccV"], window_size=window_size)
parkinson_copy[!, "AccML"] = moving_average(parkinson_copy[!, "AccML"], window_size=window_size)
parkinson_copy[!, "AccAP"] = moving_average(parkinson_copy[!, "AccAP"], window_size=window_size)

# parkinson_copy

;

In [ ]:
p = Plots.plot(collect(1:2000), parkinson_data[!, "AccV"][1:2000])
Plots.plot!(p, collect(1:2000), parkinson_copy[!, "AccV"][1:2000])

Plots.plot!(p, collect(1:2000), parkinson_data[!, "AccML"][1:2000])
Plots.plot!(p, collect(1:2000), parkinson_copy[!, "AccML"][1:2000])

Plots.plot!(p, collect(1:2000), parkinson_data[!, "AccAP"][1:2000])
Plots.plot!(p, collect(1:2000), parkinson_copy[!, "AccAP"][1:2000])

In [ ]:
CSV.write("./smoothed_filtered_data.csv", parkinson_copy)

In [ ]:
@resumable function data_loader(parkinson_dataframe, batch_size ; labels=["StartHesitation", "Turn", "Walking", "Normal"])
    pdf = deepcopy(parkinson_dataframe)

    for i in 1:batch_size:size(pdf, 1)
        if i > size(pdf, 1)
            break
        end
        x = hcat(
            pdf[!, "AccV"][i:i+batch_size],
            pdf[!, "AccML"][i:i+batch_size],
            pdf[!, "AccAP"][i:i+batch_size]
        )
    
        y = Flux.onehotbatch(pdf[!, "event"][i:i+batch_size], labels)
        
        @yield x, y
    end
end

In [ ]:
for (x, y) in data_loader(parkinson_copy, 32)
    @show x, y
    break
end

## LSTM

In [2]:
# # initialize weights with Gaussian distribution
function init_params(in::Integer, out::Integer ; mean=0.0, std=1.0)
    [
        in, out,
        rand(Truncated(Normal(mean, std), 0, 1), (out, in)), # Wf
        rand(Truncated(Normal(mean, std), 0, 1), (out, in)), # Wi
        rand(Truncated(Normal(mean, std), 0, 1), (out, in)), # Wc
        rand(Truncated(Normal(mean, std), 0, 1), (out, in)), # Wo
        rand(Truncated(Normal(mean, std), 0, 1), out), # bf
        rand(Truncated(Normal(mean, std), 0, 1), out), # bi
        rand(Truncated(Normal(mean, std), 0, 1), out), # bc
        rand(Truncated(Normal(mean, std), 0, 1), out), # bo

        # both the Long-Term and Short-Term memories are initialized with 0 values
        zeros(out), # c
        zeros(out)  # h
    ]
end

init_params (generic function with 1 method)

In [3]:
# implementing the forwarding method which is used in the Chaining process
function forward(x, lstm)
    # calculating the Memory modifier values
    f = NNlib.sigmoid_fast(lstm.Wf * x .+ lstm.bf) #
    i = NNlib.sigmoid_fast(lstm.Wi * x .+ lstm.bi) #
    o = NNlib.sigmoid_fast(lstm.Wo * x .+ lstm.bo) #

    # calculating the new memory values
    c = f .* lstm.c .+ i .* NNlib.tanh_fast(lstm.Wc * x .+ lstm.bc) # new Long-Term Memory
    h = o .* NNlib.tanh_fast(c) # new Short-Term Memory

    # updating the memory
    lstm.c, lstm.h = c, h

    # returning the hidden parameters for the next layer
    h
end

forward (generic function with 1 method)

In [4]:
# custom Long Short-Term Memory layer
mutable struct LSTM
    # input and output size of the layer
    in::Integer
    out::Integer

    Wf::AbstractMatrix # params of the Forget Gate
    Wi::AbstractMatrix # params of the Input Gate
    Wc::AbstractMatrix # params of the Input Modulation Gate
    Wo::AbstractMatrix # params of the Output Gate

    # biases of the Gates above
    bf::AbstractVector
    bi::AbstractVector
    bc::AbstractVector
    bo::AbstractVector

    # cell state (aka. long-term memory) and hidden state (aka. short-term memory)
    c::AbstractVector
    h::AbstractVector
end

In [5]:
# defining the constructor
LSTM(in::Integer, out::Integer) = LSTM(init_params(in, out)...)

LSTM

In [6]:
# Overload call, so the object can be used as a function
(lstm::LSTM)(x) = forward(x, lstm)

In [7]:
# creating a functor from the struct, so that the training can optimize its parameters
Flux.@functor LSTM

In [8]:
# creating the Long Short-Term Memory layer
function LSTM((in, out)::Pair)
    LSTM(in, out) # constructor
end

LSTM

In [13]:
# explicitely defining the trainable parameters of the layer
# all the Wrights and Biases are trainable
# exceptions >> Cell State and Hidden State
Flux.trainable(lstm::LSTM) = (lstm.Wf, lstm.Wi, lstm.Wc, lstm.Wo, lstm.bf, lstm.bi, lstm.bc, lstm.bo,)

In [14]:
m = LSTM(1 => 2)

Flux.params(m)

Params([[0.7287016229789081; 0.6659484378708097;;], [0.6489248524237079; 0.49765756180365056;;], [0.47029743932671625; 0.3272602733347646;;], [0.5466824529407139; 0.6382296846574739;;], [0.33792078551245835, 0.21388166310618117], [0.6565816767392486, 0.6831972269582973], [0.5001782461819304, 0.09487260329224652], [0.31957299010974316, 0.06945428263642425]])

In [ ]:
# function LSTM((in, out)::Pair)
#     # trainable parameters of the model
#     Wf = rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)) # params of the Forget Gate
#     Wi = rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)) # params of the Input Gate
#     Wc = rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)) # params of the Input Modulation Gate
#     Wo = rand(Truncated(Normal(mean, std), 0, 1), (hidden_size, input_size)) # params of the Output Gate

#     # biases of the Gates above
#     bf = rand(Truncated(Normal(mean, std), 0, 1), hidden_size)
#     bi = rand(Truncated(Normal(mean, std), 0, 1), hidden_size)
#     bc = rand(Truncated(Normal(mean, std), 0, 1), hidden_size)
#     bo = rand(Truncated(Normal(mean, std), 0, 1), hidden_size)

#     # cell state (aka. long-term memory) and hidden state (aka. short-term memory)
#     c = zeros(out)
#     h = zeros(out)

#     x -> forward(x, params(Wf, Wi, Wc, Wo, bf, bi, bc, bo, c, h))
# end

In [ ]:
#Flux.@functor LSTM
ltsm = x -> LTSM(x)

In [ ]:
lstm = LSTM(1 => 1)

In [ ]:
LSTM(in::Integer, out::Integer) = LSTM(init_params(in, out)...)

In [ ]:
LSTM(1, 1)

In [ ]:
(m::LSTM)(x) = forward(m, x)

In [ ]:
lstm = LSTM(10, 5)

In [ ]:
lstm(rand(10))

In [ ]:


# Flux.params(lstm)

;

In [ ]:
using LinearAlgebra

mutable struct CustomLSTM
    Wf::Matrix
    Wi::Matrix
    Wc::Matrix
    Wo::Matrix
    bf::Vector
    bi::Vector
    bc::Vector
    bo::Vector
    h::Vector
    c::Vector
end

function forward(lstm::CustomLSTM, x::Matrix)
    batch_size, sequence_length = size(x)
    hidden_size = size(lstm.h, 1)

    h = lstm.h
    c = lstm.c

    h_out = Matrix{Float64}(undef, hidden_size, sequence_length)
    c_out = Matrix{Float64}(undef, hidden_size, sequence_length)

    for t in 1:sequence_length
        # Extract the current input at time step t
        x_t = x[:, t]

        # LSTM Cell computations
        f = σ.(lstm.Wf * x_t .+ lstm.bf .+ lstm.Wf * h .+ lstm.bf)
        i = σ.(lstm.Wi * x_t .+ lstm.bi .+ lstm.Wi * h .+ lstm.bi)
        c_tilde = tanh.(lstm.Wc * x_t .+ lstm.bc .+ lstm.Wc * h .+ lstm.bc)
        c = f .* c + i .* c_tilde
        o = σ.(lstm.Wo * x_t .+ lstm.bo .+ lstm.Wo * h .+ lstm.bo)
        h = o .* tanh.(c)

        h_out[:, t] = h
        c_out[:, t] = c
    end

    lstm.h = h
    lstm.c = c

    return h_out, c_out
end